## Alocação de unidades de atendimento móvel do CBMERJ de Campos dos Goytacazes - RJ

In [2]:
# Library
import pathlib

import pandas as pd
from pulp import *
import numpy as np
import googlemaps

from utils.process import stander_text

import sys
sys.path.insert(0, str(pathlib.Path().absolute().parent))

import process.allocation

**Import data**

In [2]:
file = '../data/UNIDADES_DE_SAUDE.xlsx'
pd_dict = pd.read_excel(
    file,
    sheet_name=None # Load sheets in dataframe dictionaries 
)
# Empilhado

df = pd.concat(pd_dict.values(), ignore_index=True) 
df.head()

,ORIGEM,DESTINO,TEMPO (MIN),DISTÂNCIA (KM)
0,DBM 1/5 GUARUS,Alphaville,25 min,"12,3 km"
1,DBM 1/5 GUARUS,Baixa Grande,1h 6 min,"41,4 km"
2,DBM 1/5 GUARUS,Batista Ponto de Coqueiros,1h,"36,2 km"
3,DBM 1/5 GUARUS,Caju,16 min,"8,9 km"
4,DBM 1/5 GUARUS,Campo Limpo,48 min,"25,9 km"


In [3]:
df['EnderecoCliente']=df['DESTINO'] + ', Campos dos Goytacazes, RJ'  

## **Clean data**

Rename columns and reset index

In [4]:
df['DESTINO'] = df['DESTINO'].map(lambda x: x.strip())
df.rename(columns={'TEMPO (MIN)':'TEMPO', 'DISTÂNCIA (KM)':'DISTANCIA' }, inplace=True)

Convert str in numeric  for feature TEMPO

In [5]:
# Foi indentificado uma falta de padronização na coluna TEMPO 
df.query('TEMPO.str.contains("h")').head()

,ORIGEM,DESTINO,TEMPO,DISTANCIA,EnderecoCliente
1,DBM 1/5 GUARUS,Baixa Grande,1h 6 min,"41,4 km","Baixa Grande, Campos dos Goytacazes, RJ"
2,DBM 1/5 GUARUS,Batista Ponto de Coqueiros,1h,"36,2 km","Batista Ponto de Coqueiros, Campos dos Goytaca..."
103,DBM 1/5 GUARUS,São Martinho,1h 9 min,"45,2 km","São Martinho, Campos dos Goytacazes, RJ"
346,HOSP. TRAVESSÃO,Baixa Grande,1h 15 min,"49,4 km","Baixa Grande, Campos dos Goytacazes, RJ"
347,HOSP. TRAVESSÃO,Batista Ponto de Coqueiros,1h 7 min,"44,2 km","Batista Ponto de Coqueiros, Campos dos Goytaca..."


In [6]:
# Padronizar todos os valores para minutos   
df['TEMPO'] = df['TEMPO'].map(stander_text)

In [7]:
df.head()

,ORIGEM,DESTINO,TEMPO,DISTANCIA,EnderecoCliente
0,DBM 1/5 GUARUS,Alphaville,25,"12,3 km","Alphaville, Campos dos Goytacazes, RJ"
1,DBM 1/5 GUARUS,Baixa Grande,66,"41,4 km","Baixa Grande, Campos dos Goytacazes, RJ"
2,DBM 1/5 GUARUS,Batista Ponto de Coqueiros,60,"36,2 km","Batista Ponto de Coqueiros, Campos dos Goytaca..."
3,DBM 1/5 GUARUS,Caju,16,"8,9 km","Caju, Campos dos Goytacazes, RJ"
4,DBM 1/5 GUARUS,Campo Limpo,48,"25,9 km","Campo Limpo, Campos dos Goytacazes, RJ"


## Geocode



In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

API_KEY_GOOGLE_MAPS = os.getenv('API_KEY_GOOGLE_MAPS')

In [9]:
def get_lat_log_from_addres(add):
    '''get lat long in graus decimal thourt geocoding
    
    Args
        add (str): Adress      
    return 
        tuple: (lat, long) 
    '''
    gmaps = googlemaps.Client(key=API_KEY_GOOGLE_MAPS)
    res = gmaps.geocode(add)
    return tuple(res[0]['geometry']['location'].values())    

def get_distance_matrix(origin,destination):
    """_summary_

    Args:
        origin (tuple): tuple lat,long
        destination (tuple): tuple lat,long

    Returns:
        int: time i seconds
    """
    gmaps = googlemaps.Client(key=API_KEY_GOOGLE_MAPS)
    res = gmaps.distance_matrix(origin,destination)
    return res['rows'][0]['elements'][0]['duration']['value']

Carregar lista de endereços

In [10]:
# Endereços 
file = '../data/enderecos.csv'
df_base = pd.read_csv(file,  encoding='ISO-8859-1', delimiter=';')
df_base.head()

,Base,EnderecoBase
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt..."
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca..."
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac..."
3,HOSP. G. GUARÚS,"Avenida Senador José Carlos Pereira Pinto, 400..."
4,HOSP. SÃO JOSÉ,Estrada do Açucar sem número - Goytacazes - C...


In [11]:
# Obeter cooderndas geograficas das facilidades
if not pathlib.Path('../data/base.csv').exists():
    df_base['base_lat_long'] = df_base['EnderecoBase'].map(get_lat_log_from_addres)
    df_base.to_csv('../data/base.csv')
    
df_base = pd.read_csv('../data/base.csv', index_col=0)
df_base.head()

,Base,EnderecoBase,base_lat_long
0,5 GBM,"Rua Rui Barbosa, 863 - Centro, Campos dos Goyt...","(-21.7560766, -41.3163947)"
1,DBM 1/5 GUARUS,"Rua Alci Ferreira - Codim, Campos dos Goytaca...","(-21.709383, -41.2875301)"
2,HOSP. FERREIRA M,"Rua Rocha Leão, 02 - Centro, Campos dos Goytac...","(-21.7437715, -41.333381)"
3,HOSP. G. GUARÚS,"Avenida Senador José Carlos Pereira Pinto, 400...","(-21.7413698, -41.3129263)"
4,HOSP. SÃO JOSÉ,Estrada do Açucar sem número - Goytacazes - C...,"(-21.8294589, -41.2949733)"


In [12]:
# Obeter cooderndas geograficas dos clientes
df_cliente = df.drop_duplicates(subset=['EnderecoCliente']).copy()

In [13]:
if not pathlib.Path('../data/clientes.csv').exists():
    df_cliente['cliente_lat_long'] = df_cliente['EnderecoCliente'].map(get_lat_log_from_addres)
    df_cliente[['DESTINO', 'EnderecoCliente','cliente_lat_long']].to_csv('../data/clientes.csv')
    
df_cliente = pd.read_csv('../data/clientes.csv', index_col=0)
df_cliente.head()

,DESTINO,EnderecoCliente,cliente_lat_long
0,Alphaville,"Alphaville, Campos dos Goytacazes, RJ","(-21.7711892, -41.3106727)"
1,Baixa Grande,"Baixa Grande, Campos dos Goytacazes, RJ","(-21.9492435, -41.13520889999999)"
2,Batista Ponto de Coqueiros,"Batista Ponto de Coqueiros, Campos dos Goytaca...","(-21.9074616, -41.1586879)"
3,Caju,"Caju, Campos dos Goytacazes, RJ","(-21.7444336, -41.3375268)"
4,Campo Limpo,"Campo Limpo, Campos dos Goytacazes, RJ","(-21.8595722, -41.2413952)"


Combinar origem destino 

In [14]:
combin = []
for base in df_base.itertuples():
    combin.append(
        (
            base.Base,
            df_cliente['DESTINO'].values,
            df_cliente['cliente_lat_long'].values,
            base.base_lat_long
         )
    )

In [15]:
df_coord = pd.DataFrame(
    combin, 
    columns=['Facilidade','Clientes', 'Coord_Clientes', 'Coord_Facilidade']
).explode(['Clientes',	'Coord_Clientes'],ignore_index=True)

df_coord.drop_duplicates(inplace=True)

In [16]:
df_coord['Coord_cliente_facilidade'] = df_coord[['Coord_Clientes', 'Coord_Facilidade']].apply(lambda x:(eval(x.iloc[0]),eval(x.iloc[1])), axis=1)
df_coord.head()

,Facilidade,Clientes,Coord_Clientes,Coord_Facilidade,Coord_cliente_facilidade
0,5 GBM,Alphaville,"(-21.7711892, -41.3106727)","(-21.7560766, -41.3163947)","((-21.7711892, -41.3106727), (-21.7560766, -41..."
1,5 GBM,Baixa Grande,"(-21.9492435, -41.13520889999999)","(-21.7560766, -41.3163947)","((-21.9492435, -41.13520889999999), (-21.75607..."
2,5 GBM,Batista Ponto de Coqueiros,"(-21.9074616, -41.1586879)","(-21.7560766, -41.3163947)","((-21.9074616, -41.1586879), (-21.7560766, -41..."
3,5 GBM,Caju,"(-21.7444336, -41.3375268)","(-21.7560766, -41.3163947)","((-21.7444336, -41.3375268), (-21.7560766, -41..."
4,5 GBM,Campo Limpo,"(-21.8595722, -41.2413952)","(-21.7560766, -41.3163947)","((-21.8595722, -41.2413952), (-21.7560766, -41..."


Obter tempo base vs facilidades

In [17]:
if not pathlib.Path('../data/clientes.csv').exists():
    df_coord['Tempo'] = df_coord['Coord_cliente_facilidade'].map(lambda x: get_distance_matrix((x[0]),x[1]))
    df_coord.to_csv('../data/coordenadas.csv')

df_coord = pd.read_csv('../data/coordenadas.csv')

Pivot - matrix

In [18]:
df_matrix = df_coord.pivot(columns='Facilidade',index='Clientes',  values='Tempo' )
df_matrix.to_excel('../data/matriz_tempo.xlsx')

In [19]:
df_matrix

Facilidade,5 GBM,DBM 1/5 GUARUS,HOSP. FERREIRA M,HOSP. G. GUARÚS,HOSP. SÃO JOSÉ,HOSP. TRAVESSÃO,PS GUARUS,SANDU,UBS TOCOS,UPA,UPH BAIXA GRANDE,UPH F. SÃO THOMÉ,UPH MORRO DO COCO,UPH ST EDUARDO
Clientes,,,,,,,,,,,,,,
Aeroporto Bartolomeu Lisandro,964,300,880,783,2183,785,804,1307,3093,493,77920,2279,2364,3814
Alphaville,414,1279,733,628,829,1861,740,204,1739,991,78148,925,3440,4890
Av. Dom Bosco,673,971,363,804,1349,1534,581,698,2259,626,77783,1445,3114,4564
Av. Pelinca,593,1014,410,698,1475,1577,530,618,2385,668,77825,1571,3156,4606
Av. Rui Barbosa,283,973,602,322,1259,1555,487,351,2170,731,77888,1355,3134,4584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ururai,1425,1813,1205,1525,1889,2376,1357,1339,1881,1467,78625,1985,3955,5405
Venda Nova,1553,2418,1873,1768,1941,3000,1933,1535,2276,2130,79288,1684,4579,6029
Vila Industrial,984,231,908,803,2202,1051,824,1326,3113,521,77948,2298,2630,4080


# Modelo matemático

In [20]:
# DEFININDO VARIÁVEIS
location = df_matrix.columns.values
clients = df_matrix.index.values 
w = [1 for w in clients]
m = df_matrix.shape[0] # Nº de clientes
n = df_matrix.shape[1] # Nº de potenciais facilidades
p = 9 # Nº de facilidades   
c = df_matrix.values


In [21]:
# Define the problem
prob = LpProblem("p-median", LpMinimize)

# Define decision variables
x = LpVariable.dicts("x", [(i,j) for i in range(m) for j in range(n)], 0, 1, LpBinary)
y = LpVariable.dicts("y", [j for j in range(n)], 0, 1, LpBinary)

In [22]:
# Define objective function
prob += lpSum([c[i][j] * x[(i,j)] * w[i] for i in range(m) for j in range(n)])

In [23]:
# Define constraints
for i in range(m):
    prob += lpSum([x[(i,j)] for j in range(n)]) == 1

for i in range(m):
    for j in range(n):
        prob += x[(i,j)] <= y[j]
                 
prob += lpSum([y[j] for j in range(n)]) == p

In [24]:
# Solve the problem using branch-and-bound algorithm
prob.solve(PULP_CBC_CMD(gapRel=0.0, threads=1, timeLimit=600))

1

In [26]:
p_median = allocation.Allocation(m, n,p, df_matrix)
p_median.solver()
res = p_median.get_result()

1
Optimal objective value: 80619.0
Facility 1 is located. DBM 1/5 GUARUS
- Customer 0 is served. Aeroporto Bartolomeu Lisandro
- Customer 17 is served. Codim
- Customer 21 is served. Estr. Brejo Grande
- Customer 44 is served. Parque Aeroporto
- Customer 53 is served. Parque Boa Vista
- Customer 64 is served. Parque Eldorado
- Customer 98 is served. Parque Santos Dumont
- Customer 103 is served. Parque São Silvestre
- Customer 131 is served. Terra Prometida
- Customer 138 is served. Vila Industrial
Facility 2 is located. HOSP. FERREIRA M
- Customer 2 is served. Av. Dom Bosco
- Customer 3 is served. Av. Pelinca
- Customer 9 is served. Caju
- Customer 13 is served. Ceasa Campos do G
- Customer 25 is served. Guriri
- Customer 27 is served. Ibitioca
- Customer 35 is served. Julião Nogueira
- Customer 37 is served. Lagoa de Cima
- Customer 40 is served. Nova Brasília
- Customer 49 is served. Parque Avenida Pelinca
- Customer 60 is served. Parque Conselheiro Tomaz Coelho
- Customer 61 is ser